In [1]:
## Import Libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyodbc

In [2]:
spark = SparkSession.builder.getOrCreate() # Create Spark Session

In [3]:
# Read Data From PostgreSql 
tablename_list=['locationtime','businesstime','registeredbusiness','location']
reader = (
        spark.read.format('jdbc')
        .option("url","jdbc:postgresql://localhost:5432/businesslocation")
        .option('user','username')
        .option('password','password')
        .option("driver","org.postgresql.Driver")
)
for tablename in tablename_list:
    reader.option("dbtable",tablename).load().createTempView(tablename)

In [4]:
locationtime = spark.sql("select * from locationtime").show(5)


+-------------------+----+---+----+-----+----+--------+
|location_start_date|hour|day|week|month|year| weekday|
+-------------------+----+---+----+-----+----+--------+
|2009-11-01 00:00:00|   0|  1|  44|   11|2009|  Sunday|
|2016-05-09 00:00:00|   0|  9|  19|    5|2016|  Monday|
|2014-01-24 00:00:00|   0| 24|   4|    1|2014|  Friday|
|2007-09-24 00:00:00|   0| 24|  39|    9|2007|  Monday|
|2020-08-01 00:00:00|   0|  1|  31|    8|2020|Saturday|
+-------------------+----+---+----+-----+----+--------+
only showing top 5 rows



In [5]:
businesstime = spark.sql("select * from businesstime").show(5)

+-------------------+----+---+----+-----+----+--------+
|business_start_date|hour|day|week|month|year| weekday|
+-------------------+----+---+----+-----+----+--------+
|1973-01-02 00:00:00|   0|  2|   1|    1|1973| Tuesday|
|1968-10-01 00:00:00|   0|  1|  40|   10|1968| Tuesday|
|2000-03-06 00:00:00|   0|  6|  10|    3|2000|  Monday|
|1982-08-16 00:00:00|   0| 16|  33|    8|1982|  Monday|
|1998-01-01 00:00:00|   0|  1|   1|    1|1998|Thursday|
+-------------------+----+---+----+-----+----+--------+
only showing top 5 rows



In [6]:
registeredbusiness = spark.sql("select * from registeredbusiness").show(5)

+--------------------+-----------------------+--------------------+-----------+-----------------------+--------------------+--------------------+-------------------+-------------------+
|            uniqueid|business_account_number|      ownership_name|parking_tax|transient_occupancy_tax|   business_location|       location_name|business_start_date|location_start_date|
+--------------------+-----------------------+--------------------+-----------+-----------------------+--------------------+--------------------+-------------------+-------------------+
|0080280-05-001-00...|                  80280|Wollborg-Michelso...|      false|                  false|POINT (-122.408 3...|  Wollborg-Michelson|1973-01-02 00:00:00|2009-11-01 00:00:00|
|1077369-06-161-00...|                  22415|Pflueger & Baerwa...|      false|                  false|POINT (-122.40095...|Pflueger & Baerwa...|1968-10-01 00:00:00|2016-05-09 00:00:00|
|0030032-46-001-00...|                  30032|         Walgreen Co|   

In [7]:
location = spark.sql("select * from location").show(5)

+--------------------+--------------------+-------------+-----+--------------+
|       location_name|      street_address|         city|state|source_zipcode|
+--------------------+--------------------+-------------+-----+--------------+
|  Wollborg-Michelson|291 Geary St Ste 500|San Francisco|   CA|         94102|
|Pflueger & Baerwa...| 220 Sansome St 1090|San Francisco|   CA|         94104|
|    Walgreens #15567|       845 Market St|San Francisco|   CA|         94103|
|       Vericlaim Inc|500 Sansome St St...|San Francisco|   CA|         94111|
|Ati Restoration, Llc|25000 Industrial ...|      Hayward|   CA|         94545|
+--------------------+--------------------+-------------+-----+--------------+
only showing top 5 rows



## Create SQLServer DataBaase

In [8]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=DESKTOP-L3KHVFJ\SQLEXPRESS;""Database=;""Trusted_Connection=yes;")

In [9]:
conn = pyodbc.connect(cnxn_str,autocommit=True)
cursor = conn.cursor()
print(cursor.execute("CREATE DATABASE businesslocation"))

In [10]:
conn.close()

## Load Data To SQLServer Database

In [11]:
tablename_list=['locationtime','businesstime','registeredbusiness','location']
jdbcUrl = "jdbc:sqlserver://DESKTOP-L3KHVFJ\SQLEXPRESS:1433;database=businesslocation;trustServerCertificate=true;"

reader = (
        spark.read.format('jdbc')
        .option("url","jdbc:postgresql://localhost:5432/businesslocation")
        .option('user','username')
        .option('password','password')
        .option("driver","org.postgresql.Driver")
)



for tablename in tablename_list:
    table_reader = reader.option("dbtable",tablename).load()
    table_reader.write.format("jdbc")\
        .option("url", jdbcUrl)\
        .option("user", "username")\
        .option("password", "password")\
        .option("dbtable",tablename).save()    

In [13]:
spark.stop()